In [ ]:
from pprint import pprint
import pandas as pd
import sys, os
import random
import copy

In [ ]:
HUMAN = 1
COMP = -1

In [ ]:
def init():
  grid = int(input("Chosir votre grille : 3(3x3), 5(5x5) ou 7(7x7)"))
  game = [[0 for j in range(grid)] for i in range(grid)]
  return game, grid


In [ ]:
game, grid = init()

Chosir votre grille : 3(3x3), 5(5x5) ou 7(7x7)5


In [ ]:
df = pd.DataFrame(game)
print(format_game(df))

   0  1  2  3  4
0  .  .  .  .  .
1  .  .  .  .  .
2  .  .  .  .  .
3  .  .  .  .  .
4  .  .  .  .  .


In [ ]:
def is_empty_box(game, x, y):
  if game[x][y] == 1 or game[x][y] == -1:
    return False
  return True

In [ ]:
def is_fullfilled(game):
  for item in game:
    for elt in item:
      if elt == 0:
        return False
  return True                

In [ ]:
def format_game(df):
  df = df.replace([1],'X')
  df = df.replace([-1],'O')
  df = df.replace([0],'.')
  
  return df

In [ ]:
def check_diago_top(state, posx, posy):
    """
    """
    x = posx
    y = posy
    
        
    try:
        if x - (comboforWin-1) >= 0 and y + (comboforWin-1) and comboforWin==3:
            diagoT =  [state[x][y], state[x-1][y+1], state[x-2][y+2]]
        elif x - (comboforWin-1) >= 0 and y + (comboforWin-1) < grid and comboforWin==4:
            diagoT =  [state[x][y], state[x-1][y+1], state[x-2][y+2],state[x-3][y+3] ]
        return diagoT
    except:
        pass
    
    return 

In [ ]:
def check_the_column(state, posx, posy):
    """
    """
    column = []
    if posx + (comboforWin-1) < grid:
        for i in range(comboforWin):
            column.append(state[posx][posy])
            posx+=1
        return column
    return

In [ ]:
def check_diago_bottom(state, posx, posy):
    """
    """
    x = posx
    y = posy
    try:
        if comboforWin == 3:
            diagoB =  [state[x][y], state[x+1][y+1], state[x+2][y+2]]
        elif comboforWin == 4:
            diagoB =  [state[x][y], state[x+1][y+1], state[x+2][y+2], state[x+3][y+3]]
        return diagoB
    except:
        pass
    
    return 

In [ ]:
def check_the_line(state, posx, posy):
    """
    """
    line = []
    if posy + (comboforWin-1) < grid:
        for i in range(comboforWin):
            line.append(state[posx][posy+i])
        return line
    return

In [ ]:
def genere_params_for_win(grilles):
    if grilles == 3:
        comboforWin = 3
    else:
        comboforWin = 4
    return comboforWin

In [ ]:
comboforWin = genere_params_for_win(grid)

In [ ]:
def create_win_possibility(state):
    wins_state = []
    for x in range(len(state)):
        for y in range(len(state)):
            line = check_the_line(state, x, y)
            if line:
                wins_state.append(line)
    
            column = check_the_column(state, x, y)
            if column:
                wins_state.append(column)
            diago_bottom = check_diago_bottom(state, x, y)
            if diago_bottom:
                wins_state.append(diago_bottom)
                
            diago_top = check_diago_top(state, x, y)
            if diago_top:
                wins_state.append(diago_top)
    return wins_state

In [ ]:
def wins(state, player):
  
    win_state = create_win_possibility(state)
    if genere_params_for_win(grid) == 3 and [player, player, player] in win_state:
        return True
    elif genere_params_for_win(grid) == 4 and [player, player, player, player] in win_state:
        return True
    else:
        return False

In [ ]:
def play_naive(game):
  is_win = False
  player = 1
  count = 0
  while is_win == False:
    if count == 0:
      print(pd.DataFrame(game))

    if player == 1:
      print("Placer votre pion sur une cordonnée (x, y)")
      x = int(input("Chosir l'abscisse"))
      y = int(input("Chosir l'ordonnée"))

    if player == -1:
      x = random.randint(0, grid-1)
      y = random.randint(0, grid-1)

    if is_empty_box(game, x, y) == False:
      while is_empty_box(game, x, y) == False:
        if player == 1:
          print("Case prise, veuillez réessayer")
          x = int(input("Chosir l'abscisse"))
          y = int(input("Chosir l'ordonnée"))
        if player == -1:
          x = random.randint(0, grid-1)
          y = random.randint(0, grid-1)

    game[x][y] = player
    show_ground = copy.deepcopy(game)
    count = count + 1
    
    is_win = wins(game, player)
    if count > 0:
      #show_ground[x][y] = 'X' if player == 1 else 'O'
      df = pd.DataFrame(show_ground)
      df = format_game(df)
      print(df)

    if is_win == True:
      name = ""
      if player == 1:
        name = "Humain"
      else:
        name = "Machine"
      print("{} a gagné".format(name))
    else:
      if is_fullfilled(game) == True:
        print("Match null")

    player = -player




In [ ]:
#play_naive(game)

In [ ]:
def evaluate(state):
    """
    Perform heuristic evaluation from board.
    Heuristic - allow the computer to discover the solution
    of some problems by itself.
    """
    if wins(state, COMP):
        score = -1
    elif wins(state, HUMAN):
        score = 1
    else:
        score = 0

    return score

In [ ]:
def empty_cells(state):
    """
    Identifier à chaque étape du jeu les cellules qui reste à cocher
    """
    cells = [] # it contains all empty cells

    # Use enumerate for easy indexing
    for i, row in enumerate(state):
        for j, col in enumerate(row):
            if state[i][j] == 0:
                cells.append([i, j])

    return cells

In [ ]:
def game_over(state):
    """Check game over condition"""
    return wins(state, HUMAN) or wins(state, COMP)

In [ ]:
evaluate(game)

0

In [ ]:
def minimax(state, depth, player):
    """
    Ns implémentons `MiniMax` (Algo IA) ce qui permettra de
    déterminer d'une façon optimale les actions de `COMP`.
    Le jeu `tictactoe` peut en effet être présenté sous la forme d'un arbre. 
    L'Algo MiniMax traverse l'arbre d'une façon récursive pour trouver les meilleures combinaisons possibles.
    :return list of [best_row, best_col, best_score]
    """

    from math import inf  
    if player == COMP:
        best = [-1, -1, inf] # inf/-inf are the initial score for the players
    else:
        best = [-1, -1, -inf]

    if depth == 0 or game_over(state):
        score = evaluate(state)
        return [-1, -1, score]

    for cell in empty_cells(state):
        # Fill the empty cells with the player symbols
        x, y = cell[0], cell[1]
        state[x][y] = player
        print('\ncell : ', cell)
        print('\nsymbole : ', player)
        score = minimax(state, depth-1, -player)
        print('\nscore : ', score)
        state[x][y] = 0
        score[0], score[1] = x, y

        if player == COMP:
            if score[2] < best[2]:
                best = score
                print('\nbest : ', best)
        else:
            if score[2] > best[2]:
                best = score
                print('\nbest : ', best)

    return best


In [ ]:
def ai_turn(state):
    depth = len(empty_cells(state)) # The remaining of empty cells
    row, col, score = minimax(state, depth, COMP) # the optimal move for computer
    #state[row][col] = COMP

    return row, col
    #print("A.I Turn")
    #print(render(state))

In [ ]:
def main(game):
  is_end = False
  player = 1
  count = 0
  while is_end == False:
    if count == 0:
      print(format_game(pd.DataFrame(game)))

    if player == 1:
      print("Placer votre pion sur une cordonnée (x, y)")
      x = int(input("Chosir ligne"))
      y = int(input("Chosir colonne"))

    if player == -1:
      x, y = ai_turn(game)
      print('row col', x, y)

    if is_empty_box(game, x, y) == False:
      while is_empty_box(game, x, y) == False:
        if player == 1:
          print("Case prise, veuillez réessayer")
          x = int(input("Chosir ligne"))
          y = int(input("Chosir colonne"))
        if player == -1:
          x,y = ai_turn(game)
          print('row col', x, y)
          
    game[x][y] = player
    show_ground = copy.deepcopy(game)
    count = count + 1
    
    is_win = wins(game, player)
    if count > 0:
      df = pd.DataFrame(show_ground)
      df = format_game(df)
      print(df)

    if is_win == True:
      is_end = True
      name = ""
      if player == 1:
        name = "Humain"
      else:
        name = "Machine"
      print("{} a gagné".format(name))
    else:
      if len(empty_cells(game)) == 0:
        print("Match null")
        is_end = True

    player = -player

In [ ]:
if __name__ == '__main__':
    main(game) 

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.

cell :  [4, 0]

symbole :  -1

cell :  [3, 4]

symbole :  1

score :  [-1, -1, 0]

best :  [3, 4, 0]

score :  [3, 4, 0]

score :  [3, 4, -1]

score :  [3, 4, -1]

best :  [3, 1, -1]

cell :  [3, 4]

symbole :  -1

score :  [-1, -1, -1]

cell :  [4, 0]

symbole :  -1

cell :  [3, 1]

symbole :  1

score :  [-1, -1, 1]

best :  [3, 1, 1]

cell :  [3, 4]

symbole :  1

cell :  [3, 1]

symbole :  -1

cell :  [4, 1]

symbole :  1

score :  [-1, -1, 0]

best :  [4, 1, 0]

score :  [4, 1, 0]

best :  [3, 1, 0]

cell :  [4, 1]

symbole :  -1

score :  [-1, -1, -1]

best :  [4, 1, -1]

score :  [4, 1, -1]

cell :  [4, 1]

symbole :  1

cell :  [3, 1]

symbole :  -1

cell :  [3, 4]

symbole :  1

score :  [-1, -1, 0]

best :  [3, 4, 0]

score :  [3, 4, 0]

best :  [3, 1, 0]

cell :  [3, 4]

symbole :  -1

score :  [-1, -1, -1]

best :  [3, 4, -1]

score :  [3, 4, -1]

score :  [3, 1, 1]

cell :  [4, 1]

symbole :  -1

KeyboardInterrupt: ignored

In [ ]:
minimax(game, 4, 1)